In [14]:
import numpy as np
import pandas as pd
import os
import data_processing as dp
from sklearn.preprocessing import StandardScaler
import time
import h5py

In [ ]:
# Summarize labels in each file and save them to csv file

df = pd.DataFrame(index=[0,1,2,3,4,5,6])
for dirname,_,filenames in os.walk('E:\Document\教材\Master thesis\data'):
    for filename in filenames:
        dir_depth = len(dirname.split(os.path.sep))
        if dir_depth == 5:
            if filename.endswith('.csv'):
                file = os.path.join(dirname,filename)
                emg_data = pd.read_csv(file)
                try:
                    emg_data.Label2
                except AttributeError:
                    continue
                else:
                    info = emg_data.Label2.value_counts()
                    df[filename] = info
                print(info)
            #print(os.path.join(dirname,filename))
df.to_csv('whole_data_info.csv',index=True)
#df.to_csv('whole_data_info.txt',sep='\t',index=True)

In [27]:
emg_data = pd.read_csv('.\data\P812_M050_2_B_FoG_trial_2_emg.csv')

In [40]:
# read file name of data with various Labels
df = pd.read_csv('./useful_data_label.csv',index_col=0) 
# read file name of data with only label 0
df2 = pd.read_csv('./unuseful_data_label.csv',index_col=0)
# read some of the data with only label 0
df3 = pd.read_csv('./data/file_name.txt',header=None)

In [41]:
np.concatenate([np.array(df.columns),np.array('正常/'+df3.loc[:,0])])

array(['G04_FoG_trial_1_emg.csv', 'G04_FoG_trial_2_emg.csv',
       'G06_FoG_trial_1_emg.csv', 'G06_FoG_trial_2_emg.csv',
       'G06_FoG_trial_3_emg.csv', 'G07_Freezing_Trial1_trial_1_emg.csv',
       'G08_FoG_1_trial_1_emg.csv', 'G08_FoG_2_trial_1_emg.csv',
       'G11_FoG_trial_1_emg.csv', 'G11_FoG_trial_2_emg.csv',
       'P379_M050_2_OFF_A_FoG_trial_1_emg.csv',
       'P379_M050_2_OFF_A_FoG_trial_2_emg.csv',
       'P379_M050_2_OFF_A_FoG_trial_3_emg.csv',
       'P379_M050_2_OFF_B_FoG_trial_1_emg.csv',
       'P379_M050_2_OFF_B_FoG_trial_2_emg.csv',
       'P379_M050_2_OFF_B_FoG_trial_3_emg.csv',
       'P551_M050_2_A_FoG_trial_1_emg.csv',
       'P551_M050_2_B_FoG_trial_1_emg.csv',
       'P551_M050_2_B_FoG_trial_2_emg.csv',
       'P812_M050_2_B_FoG_trial_1_emg.csv',
       'P812_M050_2_B_FoG_trial_2_emg.csv',
       '正常/G02_Walking_trial_1_emg.csv', '正常/G03_Walking_trial_1_emg.csv',
       '正常/G03_Walking_trial_2_emg.csv',
       '正常/G05_Walking_struct_fixed_trial_1_emg.csv',
 

In [13]:
width = 256
stride = 64
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [20]:
import imp
imp.reload(dp)

<module 'data_processing' from 'E:\\Document\\jupyter\\Master Thesis\\data_processing.py'>

In [58]:

files = np.concatenate([np.array(df.columns),np.array('正常/'+df3.loc[:,0])])
widths = [128,256]

for width in widths:
    features = []
    labels = []
    stride = width//8
    for file in files:
        feature,label = dp.pipeline_feature('./data/'+file,width = width, 
                                           stride=stride,
                                           threshold_WAMP = threshold_WAMP,
                                           threshold_SSC = threshold_SSC,
                                           bins=bins,
                                           ranges=HIST_range,
                                           show_para=False)
        features += feature.tolist()
        labels += label.tolist()
    print('save...')
    print(stride)
    save_file = './processed data/features_W%d_S%d.hdf5'%(width,stride)
    with h5py.File(save_file,'w') as f:
        f.create_dataset('features',data=np.array(features),
                         chunks = True,dtype = np.float32)
        f.create_dataset('labels',data=np.array(labels), dtype = np.int8)
    print('save finished.')

save...
16
save finished.
save...
32
save finished.


In [35]:
np.array(features).shape

(2325, 168)

In [4]:
num = df.sum(axis=1)

In [5]:
(num-256)/32

0    14576.34375
1     9322.81250
2    15632.78125
3     1228.59375
4       65.81250
5       -8.00000
6     7140.87500
dtype: float64

In [54]:
width = 256
stride = 64
threshold_WAMP = 30
threshold_ZC = 0
threshold_SSC = 1
bins=9
bound = 70
HIST_range = (-bound,bound)

In [51]:
# read the data and labels of df

X = []
Y = []
#sc = StandardScaler(with_mean=False)
i = 0
start = time.time()
for file in df.columns:
    emg_data = pd.read_csv('./data/'+file)
    emg_data = emg_data.fillna({'LEFT_TA':emg_data.LEFT_TA.mean(),
                                'LEFT_TS':emg_data.LEFT_TS.mean(),
                                'LEFT_BF':emg_data.LEFT_BF.mean(),
                                'LEFT_RF':emg_data.LEFT_RF.mean(),
                                'RIGHT_TA':emg_data.RIGHT_TA.mean(),
                                'RIGHT_TS':emg_data.RIGHT_TS.mean(),
                                'RIGHT_BF':emg_data.RIGHT_BF.mean(),
                                'RIGHT_RF':emg_data.RIGHT_RF.mean()})
    #emg_data = emg_data[emg_data.Label1 == emg_data.Label2].reset_index(drop=True)
    x,y = dp.generate_window_slide_data(emg_data,width=width,stride=stride,scaler=False)
    X += x.tolist()
    Y += y.tolist()
    i += 1
    print('i:%d'%i,x.shape)
end = time.time()
duration = end-start
print('Duration: %f'%(duration))

i:1 (1072, 256, 8)
i:2 (1088, 256, 8)
i:3 (530, 256, 8)
i:4 (474, 256, 8)
i:5 (470, 256, 8)
i:6 (2334, 256, 8)
i:7 (3253, 256, 8)
i:8 (5280, 256, 8)
i:9 (425, 256, 8)
i:10 (310, 256, 8)
i:11 (363, 256, 8)
i:12 (362, 256, 8)
i:13 (316, 256, 8)
i:14 (409, 256, 8)
i:15 (379, 256, 8)
i:16 (374, 256, 8)
i:17 (390, 256, 8)
i:18 (1309, 256, 8)
i:19 (1023, 256, 8)
i:20 (1097, 256, 8)
i:21 (1016, 256, 8)
Duration: 25.108637


In [52]:
# read the data and labels of df2 or df3

ind = df2.iloc[1].isna()
features = []
#sc = StandardScaler(with_mean=False)
#width = 256
#stride = 32
start = time.time()
i = 0
for file in np.array(df3.loc[:,0]):
    #if file in np.array(df3.loc[:,0]):
    #    continue
    emg_data = pd.read_csv('./data/正常/'+file)
    emg_data = emg_data.fillna({'LEFT_TA':emg_data.LEFT_TA.mean(),
                                'LEFT_TS':emg_data.LEFT_TS.mean(),
                                'LEFT_BF':emg_data.LEFT_BF.mean(),
                                'LEFT_RF':emg_data.LEFT_RF.mean(),
                                'RIGHT_TA':emg_data.RIGHT_TA.mean(),
                                'RIGHT_TS':emg_data.RIGHT_TS.mean(),
                                'RIGHT_BF':emg_data.RIGHT_BF.mean(),
                                'RIGHT_RF':emg_data.RIGHT_RF.mean()})
    #emg_data = emg_data[emg_data.Label1 == emg_data.Label2].reset_index(drop=True)
    x,y = dp.generate_window_slide_data(emg_data,width=width,stride=stride,scaler=False)
    #feature = dp.generate_feature(x,threshold_WAMP=threshold_WAMP,
    #                          threshold_ZC=threshold_ZC,
    #                          threshold_SSC=threshold_SSC,
    #                          bins=bins,ranges=HIST_range)
    #features += feature.tolist()
    X += x.tolist()
    Y += y.tolist()
    i += 1
    print('i:%d'%i,x.shape)
    
end = time.time()
duration = end-start
print('Duration: %f'%(duration))

i:1 (506, 256, 8)
i:2 (703, 256, 8)
i:3 (588, 256, 8)
i:4 (1040, 256, 8)
i:5 (699, 256, 8)
i:6 (1335, 256, 8)
i:7 (895, 256, 8)
i:8 (958, 256, 8)
i:9 (844, 256, 8)
i:10 (727, 256, 8)
i:11 (768, 256, 8)
i:12 (814, 256, 8)
i:13 (833, 256, 8)
i:14 (690, 256, 8)
i:15 (761, 256, 8)
i:16 (1198, 256, 8)
i:17 (1330, 256, 8)
i:18 (1028, 256, 8)
i:19 (803, 256, 8)
i:20 (1021, 256, 8)
i:21 (1935, 256, 8)
i:22 (1128, 256, 8)
i:23 (818, 256, 8)
i:24 (1299, 256, 8)
i:25 (707, 256, 8)
i:26 (732, 256, 8)
Duration: 26.938807


In [19]:
ind = df2.iloc[1].isna()
df2.columns[ind]

Index(['G02_Walking_trial_1_emg.csv', 'G03_Walking_trial_1_emg.csv',
       'G03_Walking_trial_2_emg.csv',
       'G05_Walking_struct_fixed_trial_1_emg.csv',
       'G05_Walking_struct_fixed_trial_2_emg.csv',
       'G05_Walking_struct_fixed_trial_3_emg.csv', 'G09_FoG_trial_1_emg.csv',
       'G09_FoG_trial_2_emg.csv', 'G09_FoG_trial_3_emg.csv',
       'G09_Walking_trial_2_emg.csv',
       ...
       'P940_M100_B_Walking_2_trial_2_emg.csv',
       'P940_M100_B_Walking_2_trial_6_emg.csv',
       'P940_MSham_A_FoG_trial_1_emg.csv', 'P940_MSham_A_FoG_trial_3_emg.csv',
       'P940_MSham_A_Walking_trial_2_emg.csv',
       'P940_MSham_A_Walking_trial_4_emg.csv',
       'P940_MSham_A_Walking_trial_6_emg.csv',
       'P940_MSham_B_Walking_trial_2_emg.csv',
       'P940_MSham_B_Walking_trial_4_emg.csv',
       'P940_MSham_B_Walking_trial_6_emg.csv'],
      dtype='object', length=153)

In [53]:
pd.Series(Y).value_counts()


0    30980
2     7223
1     4278
6     3401
3      524
4       28
dtype: int64

In [8]:
np.array(Y).shape

(46434,)

In [9]:
file = './processed data/data_set_after_window_S64_withoutSC_allPa.hdf5'
with h5py.File(file,'w') as f:
    f.create_dataset('cwt_data',data=np.array(X),
                     chunks = True,dtype = np.float32)
    f.create_dataset('label2',data=np.array(Y), dtype = np.int8)

In [38]:
file = './processed data/feature_restdata.hdf5'
with h5py.File(file,'w') as f:
    f.create_dataset('features',data=np.array(features),
                     chunks = True,dtype = np.float32)
    f.create_dataset('labels',data=np.array(Y), dtype = np.int8)

In [31]:
emg_data[emg_data.Label1 == emg_data.Label2].Label1.value_counts()

0    26775
2    22889
1     2842
3      761
Name: Label1, dtype: int64

In [16]:
pd.Series(Y).value_counts()

2    14429
0    13629
1     8567
6     6797
3     1051
4       57
dtype: int64

In [17]:
path = './processed data/data_set_after_window_label12.hdf5'
with h5py.File(path,'r') as f:
    x = f['cwt_data'][...]
    y = f['label2'][...]

In [18]:
pd.Series(y).value_counts()

2    11903
0    11711
6     5887
1     5405
3       92
dtype: int64

In [3]:
pd.read_csv('./data/file_name.txt',header=None)

,0
0,G02_Walking_trial_1_emg.csv
1,G03_Walking_trial_1_emg.csv
2,G03_Walking_trial_2_emg.csv
3,G05_Walking_struct_fixed_trial_1_emg.csv
4,G05_Walking_struct_fixed_trial_3_emg.csv
5,G09_FoG_trial_1_emg.csv
6,G09_FoG_trial_3_emg.csv
7,G11_Walking_trial_2_emg.csv
8,P231_M050_A_Walking_trial_2_emg.csv
9,P231_M050_A_Walking_trial_4_emg.csv
